In [2]:
import numpy as np
from pupil_apriltags import Detector
import cv2
from time import sleep, time
from IPython.display import clear_output
from picamera.array import PiYUVArray, PiRGBArray
from picamera import PiCamera
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
camera_info = {}
#Camera Resolution
camera_info["res"] = (1280, 760)
#Camera Intrinsic Matrix (3x3)
camera_info["K"] = np.array(
    [[631.6058624841243, 0.0, 673.9002987027918], [0.0, 627.4303222760955, 380.85431690312384], [0.0, 0.0, 1.0]])
#The non-default elements of the K array, in the AprilTag specification
camera_info["params"] = [631.605, 627.43, 673.9, 380.85]
#Fisheye Camera Distortion Matrix
camera_info["D"] = np.array(
    [[-0.031080677599846774], [-0.006061559072085696], [-0.0011641369792770276], [0.00028577486827623653]])
#Fisheye flag
camera_info["fisheye"] = True
map1, map2 = cv2.fisheye.initUndistortRectifyMap(camera_info["K"], camera_info["D"], np.eye(3),
                                                         camera_info["K"], camera_info["res"], cv2.CV_16SC2)
camera_info["map_1"]=map1
camera_info["map_2"]=map2

In [8]:
camera_info = {}
#Camera Resolution
camera_info["res"] = (640, 480)
#Camera Intrinsic Matrix (3x3)
camera_info["K"] = np.array([[313.11130800756115, 0.0, 336.11351317641487], [0.0, 310.34427179740504, 239.24222723346466], [0.0, 0.0, 1.0]])
#The non-default elements of the K array, in the AprilTag specification
camera_info["params"] = [313.111, 310.344, 336.114, 239.242]
#Fisheye Camera Distortion Matrix
camera_info["D"] = np.array([[-0.03574382363559852], [0.0028133336786254765], [-0.007814648102960479], [0.003381442340208307]])
#Fisheye flag
camera_info["fisheye"] = True
map1, map2 = cv2.fisheye.initUndistortRectifyMap(camera_info["K"], camera_info["D"], np.eye(3),
                                                         camera_info["K"], camera_info["res"], cv2.CV_16SC2)
camera_info["map_1"]=map1
camera_info["map_2"]=map2

In [42]:
families = "tagStandard52h13"
#Size of Tag in Meters
tag_size = .04
detector = Detector(families=families,nthreads=4)


### To Do Tomorrow

- Figure out why undistort2 doesnt work, see if its faster once working
- Get everything else faster as well

### Timing Dec 16

Get Image from buffer: 0.0010760211944580078 <br>
Undistort: 0.17095403671264647 <br>
Apriltag: 0.02233040809631348 <br>
Other: 0.0008922672271728382 <br>
Total: 0.5627192783355712 <br>


### Notes Dec 17
Switched to 640x480 becuase 10 FPS max with 1280x768
60 FPS with quad-decimate 4, no detections over a meter though
30 FPS with no-quad decimate, better detection distance
20 FPS with more than 2 tags
nthreads=4 adds 1 fps

In [47]:
RUN_TIMER = 30
positions = np.zeros(shape=(3, 1))
first_frame = True
try:
    camera = PiCamera(sensor_mode=7,resolution=camera_info["res"])
    rawCapture = PiYUVArray(camera, size=camera_info["res"])
    read=[]
    undist=[]
    detect=[]
    end=[]
    full=[]
    t0 = time()
    for f in camera.capture_continuous(rawCapture, format="yuv", use_video_port=True):
        t1=time()
        if first_frame:
            first_frame = False
            # Reset the buffer for the next image
            rawCapture.truncate(0)
            continue
        # Stop after RUN_TIMER seconds
        if (time() - t0) > RUN_TIMER:
            break

        # Get the intensity component of the image (a trick to get black and white images)
        I_distorted = f.array[:, :, 0]
        rawCapture.truncate(0)
        read.append(time()-t1)
        I= cv2.remap(I_distorted, camera_info["map_1"], camera_info["map_2"], interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT) 
        undist.append(time()-t1)
        detected_tags = detector.detect(I, estimate_tag_pose=True, camera_params=camera_info["params"],
                                                     tag_size=tag_size)
        detect.append(time()-t1)
        tmp_poses = [tag.pose_t for tag in detected_tags]
        if tmp_poses:
            avg=np.mean(np.concatenate(tmp_poses, axis=1), axis=1, keepdims=True)
            positions = np.hstack((positions, avg))
            print(f"Time:{time() - t0:.3f}, X:{avg[0,0]:.3f},Y:{avg[1,0]:.3f},X:{avg[2,0]:.3f}")
        else:
            # If no new tag detections in this frame, assume (naively) that the camera stayed in the same position
            # as the previous frame
            positions = np.hstack((positions, positions[:, -1:]))
            print("No detections")
        end.append(time()-t1)
        full.append(time()-t0)
    print(positions.shape)
    print(f"Time to Read:{np.mean(read)}")
    print(f"Time to Undistort:{np.mean(undist)-np.mean(read)}")
    print(f"Time to Detect:{np.mean(detect)-np.mean(undist)}")
    print(f"Time to End of Loop:{np.mean(end)-np.mean(detect)}")
    print(f"Total Time:{np.mean(end)}")
    print(f"Estimated FPS:{1/np.mean(end)}")
    print(f"Total FPS Processed: {positions.shape[1]/RUN_TIMER}")
    camera.close()
    rawCapture.close()
except Exception as e:
    print(e)
    rawCapture.close()
    camera.close()

Time:0.183, X:-0.048,Y:0.071,X:0.404
Time:0.272, X:-0.045,Y:0.065,X:0.401
Time:0.365, X:-0.041,Y:0.047,X:0.398
Time:0.470, X:-0.039,Y:0.024,X:0.392
Time:0.561, X:-0.036,Y:-0.008,X:0.389
Time:0.628, X:-0.034,Y:-0.023,X:0.389
Time:0.705, X:-0.033,Y:-0.025,X:0.375
Time:0.784, X:-0.028,Y:-0.011,X:0.344
Time:0.850, X:-0.027,Y:-0.011,X:0.333
Time:0.908, X:-0.029,Y:-0.014,X:0.326
Time:0.994, X:-0.032,Y:-0.015,X:0.321
Time:1.056, X:-0.031,Y:-0.012,X:0.310
Time:1.130, X:-0.031,Y:-0.010,X:0.306
Time:1.204, X:-0.031,Y:-0.010,X:0.305
Time:1.298, X:-0.030,Y:-0.013,X:0.295
Time:1.360, X:-0.029,Y:-0.013,X:0.293
Time:1.446, X:-0.027,Y:-0.012,X:0.283
Time:1.505, X:-0.027,Y:-0.011,X:0.278
Time:1.585, X:-0.025,Y:-0.009,X:0.268
Time:1.660, X:-0.022,Y:-0.007,X:0.256
Time:1.717, X:-0.021,Y:-0.006,X:0.249
Time:1.799, X:-0.021,Y:-0.007,X:0.242
Time:1.881, X:-0.021,Y:-0.008,X:0.236
Time:1.937, X:-0.022,Y:-0.007,X:0.228
Time:2.017, X:-0.022,Y:-0.006,X:0.224
Time:2.124, X:-0.022,Y:-0.003,X:0.221
Time:2.188, X:-0

Time:17.630, X:-0.023,Y:-0.033,X:0.475
Time:17.707, X:-0.014,Y:-0.034,X:0.484
Time:17.783, X:-0.002,Y:-0.033,X:0.502
Time:17.857, X:0.007,Y:-0.031,X:0.490
Time:17.935, X:0.018,Y:-0.031,X:0.506
Time:18.008, X:0.023,Y:-0.030,X:0.505
Time:18.096, X:0.031,Y:-0.024,X:0.499
Time:18.170, X:0.033,Y:-0.020,X:0.492
Time:18.245, X:0.030,Y:-0.018,X:0.477
Time:18.315, X:0.028,Y:-0.018,X:0.471
Time:18.381, X:0.025,Y:-0.019,X:0.457
Time:18.440, X:0.023,Y:-0.020,X:0.457
Time:18.522, X:0.019,Y:-0.021,X:0.445
Time:18.614, X:0.016,Y:-0.020,X:0.442
Time:18.682, X:0.012,Y:-0.019,X:0.423
Time:18.750, X:0.007,Y:-0.020,X:0.419
Time:18.827, X:0.001,Y:-0.020,X:0.415
Time:18.896, X:-0.002,Y:-0.019,X:0.406
Time:18.967, X:-0.006,Y:-0.017,X:0.396
Time:19.035, X:-0.008,Y:-0.016,X:0.390
Time:19.095, X:-0.011,Y:-0.016,X:0.384
Time:19.159, X:-0.015,Y:-0.017,X:0.384
Time:19.236, X:-0.018,Y:-0.019,X:0.383
Time:19.314, X:-0.023,Y:-0.024,X:0.388
Time:19.384, X:-0.026,Y:-0.025,X:0.392
Time:19.459, X:-0.026,Y:-0.025,X:0.396


In [45]:
1/np.mean(np.diff(full))

11.720933767910875